In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import *
import random



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('faqs.json', encoding="utf-8").read()

## Parsing the valid JSON data to python dictionaries 
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

88 documents
64 classes ['Amyotrophic Lateral Sclerosis', 'Are COVID-19 vaccines safe for people living with HIV', 'Are antibiotics effective in preventing or treating the COVID-19', 'Are people with tuberculosis likely to be at increased risk of COVID-19 infection, illness and death', 'Are pools and water areas safe to use', 'Are there any medicines or therapies that can prevent or cure COVID-19', 'Are there precautions to take while eating in a restaurant', 'As a smoker, am I likely to get more severe symptoms if infected', 'As a smoker, is my risk of getting the COVID-19 virus higher than that of a non-smoker', 'Asthma', 'COVID-19 (Coronavirus) & COPD', 'COVID-19 (Coronavirus) & Cancer', 'COVID-19 (Coronavirus) & Cystic Fibrosis ', 'COVID-19 (Coronavirus) and Diabetes', 'Can I have the second dose with a different vaccine than the first dose', 'Can we stop taking precautions after being vaccinated', 'Can you get the virus from people who were in the room previous to you', 'Dementia'

In [5]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-5-3544dc409e53>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [6]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(optimizer='adam', loss='mean_squared_error')

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/500
18/18 [==============================] - 32s 3ms/step - loss: 0.0154
Epoch 2/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0154
Epoch 3/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0153
Epoch 4/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0153
Epoch 5/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0153
Epoch 6/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0153
Epoch 7/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0152
Epoch 8/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0152
Epoch 9/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0151
Epoch 10/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0151
Epoch 11/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0147
Epoch 12/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0146
Epoch 13/500
18/18 [================

18/18 [==============================] - 0s 5ms/step - loss: 0.0034
Epoch 103/500
18/18 [==============================] - 0s 5ms/step - loss: 0.0034
Epoch 104/500
18/18 [==============================] - 0s 5ms/step - loss: 0.0034
Epoch 105/500
18/18 [==============================] - 0s 5ms/step - loss: 0.0040
Epoch 106/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0026
Epoch 107/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0017
Epoch 108/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0025
Epoch 109/500
18/18 [==============================] - 0s 5ms/step - loss: 0.0028
Epoch 110/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0030
Epoch 111/500
18/18 [==============================] - 0s 5ms/step - loss: 0.0025
Epoch 112/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0028
Epoch 113/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0025
Epoch 114/500
18/18 [=========

Epoch 202/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0025
Epoch 203/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0016
Epoch 204/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0018
Epoch 205/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0010
Epoch 206/500
18/18 [==============================] - 0s 4ms/step - loss: 6.7693e-04
Epoch 207/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0023
Epoch 208/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0021
Epoch 209/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0017
Epoch 210/500
18/18 [==============================] - 0s 4ms/step - loss: 7.8245e-04
Epoch 211/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0014
Epoch 212/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0018
Epoch 213/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0019
Epoch 21

18/18 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 302/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0014
Epoch 303/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0014
Epoch 304/500
18/18 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 305/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 306/500
18/18 [==============================] - 0s 3ms/step - loss: 3.6889e-04
Epoch 307/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 308/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 309/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0021
Epoch 310/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 311/500
18/18 [==============================] - 0s 2ms/step - loss: 8.0609e-04
Epoch 312/500
18/18 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 313/500
18/18 [=

18/18 [==============================] - 0s 4ms/step - loss: 7.7884e-04
Epoch 400/500
18/18 [==============================] - 0s 4ms/step - loss: 9.9956e-04
Epoch 401/500
18/18 [==============================] - 0s 4ms/step - loss: 8.1335e-04
Epoch 402/500
18/18 [==============================] - 0s 4ms/step - loss: 6.3135e-04
Epoch 403/500
18/18 [==============================] - 0s 4ms/step - loss: 8.7794e-04
Epoch 404/500
18/18 [==============================] - 0s 4ms/step - loss: 6.3458e-04
Epoch 405/500
18/18 [==============================] - 0s 4ms/step - loss: 6.8417e-04
Epoch 406/500
18/18 [==============================] - 0s 4ms/step - loss: 6.8800e-04
Epoch 407/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0013
Epoch 408/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0011
Epoch 409/500
18/18 [==============================] - 0s 4ms/step - loss: 5.4765e-04
Epoch 410/500
18/18 [==============================] - 0s 4ms/step - loss: 9

18/18 [==============================] - 0s 4ms/step - loss: 7.0442e-04
Epoch 497/500
18/18 [==============================] - 0s 4ms/step - loss: 3.4282e-04
Epoch 498/500
18/18 [==============================] - 0s 4ms/step - loss: 0.0017
Epoch 499/500
18/18 [==============================] - 0s 5ms/step - loss: 8.1449e-04
Epoch 500/500
18/18 [==============================] - 0s 5ms/step - loss: 7.2394e-04
model created


In [7]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('faqs.json', encoding="utf-8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [9]:

# Importing the libraries
from tkinter import *
from tkinter import messagebox                           
import os            
import numpy as np
import pandas as pd
   
# Importing the dataset
training_dataset = pd.read_csv('symptom_data.csv')

# Slicing and Dicing the dataset to separate features from predictions
X = training_dataset.iloc[:, 0:25].values
#print(X)
y = training_dataset.iloc[:, -1].values
#print(y)

# Dimensionality Reduction/ reducing the input features for removing redundancies
dimensionality_reduction = training_dataset.groupby(training_dataset['results']).max()
#print(dimensionality_reduction)

# Encoding String values to integer constants
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)
#print(y)

# Splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Implementing the Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

# Saving the information of columns
cols     = training_dataset.columns
cols     = cols[:-1]


# Checking the Important features
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

# Implementing the Visual Tree
from sklearn.tree import _tree

# Method to simulate the working of a Chatbot by extracting and formulating questions
def print_disease(node):
        #print(node)
        node = node[0]
        
        val  = node.nonzero() 
  
        disease = labelencoder.inverse_transform(val[0])
        return disease
def recurse(node, depth):
            global val,ans
            global tree_,feature_name,symptoms_present
            indent = "  " * depth
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                yield name + " "
                

                ans = ans.lower()
                if ans == 'yes':
                    val = 1
                else:
                    val = 0
                if  val <= threshold:
                    yield from recurse(tree_.children_left[node], depth + 1)
                else:
                    symptoms_present.append(name)
                    yield from recurse(tree_.children_right[node], depth + 1)
            else:
                strData=""
                
                present_disease = float(print_disease(tree_.value[node]))

                if present_disease==1.0:
                    strData="\n\n\n Result: You may have Covid.\n Please get tested.\n\n Press Clear and Start for new diagnosis"
                else:
                    strData='\n\n\n Result: You have a very less chance of being infected with Covid. Stay Safe.\n\n Press Clear and Start for new diagnosis'
               
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
                
                red_cols = dimensionality_reduction.columns 
                symptoms_given = red_cols[dimensionality_reduction.loc[present_disease].values[0].nonzero()]

                hyperlink = HyperlinkManager(QuestionDigonosis.objRef.txtDigonosis)
                def click1():
                    webbrowser.open_new(str(row['link'].values[0]))
                      
                
                
        
def tree_to_code(tree, feature_names):
        global tree_,feature_name,symptoms_present
        tree_ = tree.tree_
        #print(tree_)
        feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]
      
        symptoms_present = []   


def execute_bot():

    tree_to_code(classifier,cols)


class GUI():
    objRef1=None
    
    def __init__(self):
        base = Tk()
        base.title("Chatbot")
        base.geometry("400x500")
        base.resizable(width=FALSE, height=FALSE)
        GUI.objRef1=self
        

        #Create Chat window
        self.ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

        self.ChatLog.config(state=DISABLED)

        #Bind scrollbar to Chat window
        scrollbar = Scrollbar(base, command=self.ChatLog.yview, cursor="heart")
        self.ChatLog['yscrollcommand'] = scrollbar.set

        #Create Button to send message
        SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                        bd=0, bg="light blue", activebackground="light blue",fg='#ffffff',
                        command= self.send )

        #Create the box to enter message
        self.EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
        #EntryBox.bind("<Return>", send)


        #Place all components on the screen
        scrollbar.place(x=376,y=6, height=386)
        self.ChatLog.place(x=6,y=6, height=386, width=370)
        self.EntryBox.place(x=128, y=401, height=90, width=265)
        SendButton.place(x=6, y=401, height=90)

    def send(self):
        msg = self.EntryBox.get("1.0",'end-1c').strip()
        self.EntryBox.delete("0.0",END)

        if msg != '':
            self.ChatLog.config(state=NORMAL)
            self.ChatLog.insert(END, "You: " + msg + '\n\n')
            self.ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

            res = chatbot_response(msg)
            self.ChatLog.insert(END, "Bot: " + res + '\n\n')

            self.ChatLog.config(state=DISABLED)
            self.ChatLog.yview(END)


class QuestionDigonosis(Frame):
    objIter=None
    objRef=None
    
     
        
    def __init__(self,master=None):
        master.title("Faq Bot")
       
        master.state("z")

        QuestionDigonosis.objRef=self
        super().__init__(master=master)
        self["bg"]="light blue"
        self.createWidget()
        self.iterObj=None
    
     
        
    def createWidget(self):
        self.lblQuestion=Label(self,text="Question",width=12,bg="bisque")
        self.lblQuestion.grid(row=0,column=0,rowspan=4)

        self.lblDigonosis = Label(self, text="Digonosis",width=12,bg="bisque")
        self.lblDigonosis.grid(row=4, column=0,sticky="n",pady=5)

       
        self.txtQuestion = Text(self, width=100,height=4)
        self.txtQuestion.grid(row=0, column=1,rowspan=4,columnspan=20)

        self.varDiagonosis=StringVar()
        self.txtDigonosis =Text(self, width=100,height=14)
        self.txtDigonosis.grid(row=4, column=1,columnspan=20,rowspan=20,pady=5)

        self.btnNo=Button(self,text="No",width=12,bg="bisque", command=self.btnNo_Click)
        self.btnNo.grid(row=25,column=0)
        self.btnYes = Button(self, text="Yes",width=12,bg="bisque", command=self.btnYes_Click)
        self.btnYes.grid(row=25, column=1,columnspan=20,sticky="e")

        self.btnClear = Button(self, text="Clear",width=12,bg="bisque", command=self.btnClear_Click)
        self.btnClear.grid(row=27, column=0)
        self.btnStart = Button(self, text="Start",width=12,bg="bisque", command=self.btnStart_Click)
        self.btnStart.grid(row=27, column=1,columnspan=20,sticky="e")
        
        self.btnChatBot=Button(self,text="Chat Bot",width=12,bg="bisque", command=self.btnChatBot_Click)
        self.btnChatBot.grid(row=30, column=1,columnspan=20,sticky="e")
        
        
       
    def btnNo_Click(self):
        global val,ans
        global val,ans
        ans='no'
        str1=QuestionDigonosis.objIter.__next__()
        self.txtQuestion.delete(0.0,END)
        self.txtQuestion.insert(END,str1+"\n")
        
    def btnYes_Click(self):
        global val,ans
        ans='yes'
        str1=QuestionDigonosis.objIter.__next__()
        self.txtQuestion.delete(0.0,END)
        self.txtQuestion.insert(END,str1+"\n")
        
    def btnClear_Click(self):
        self.txtDigonosis.delete(0.0,END)
        self.txtQuestion.delete(0.0,END)
    def btnStart_Click(self):
        execute_bot()
        self.txtDigonosis.delete(0.0,END)
        self.txtQuestion.delete(0.0,END)
        self.txtDigonosis.insert(END,"Please Click on Yes or No for the Above symptoms in Question")                  
        QuestionDigonosis.objIter=recurse(0, 1)
        str1=QuestionDigonosis.objIter.__next__()
        self.txtQuestion.insert(END,str1+"\n")
    
    def btnChatBot_Click(self):
        o1=GUI()
        
        mainloop()



class MainForm(Frame):
    main_Root = None
    def destroyPackWidget(self, parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        MainForm.main_Root = master
        super().__init__(master=master)
        master.geometry("300x250")
        master.title("Account Login")
        self.createWidget()
    def createWidget(self):
        self.lblMsg=Label(self, text="COVID-19 HelpBot", bg="PeachPuff2", width="300", height="2", font=("Helvetica", 18,"bold italic"))
        self.lblMsg.pack()
        self.btnLogin=Button(self, text="Login", height="2", width="300", command = self.lblLogin_Click)
        self.btnLogin.pack()
        self.btnRegister=Button(self, text="Register", height="2", width="300", command = self.btnRegister_Click)
        self.btnRegister.pack()
        self.lblTeam=Label(self, text="Made by:", bg="PeachPuff2", width = "250", height = "1", font=("Times", 18))
        self.lblTeam.pack()
        self.lblTeam1=Label(self, text="Deepansha Adlakha", bg="light blue", width = "250", height = "1", font=("Times", 18,"bold italic"))
        self.lblTeam1.pack()
        
    def lblLogin_Click(self):
        self.destroyPackWidget(MainForm.main_Root)
        frmLogin=Login(MainForm.main_Root)
        frmLogin.pack()
    def btnRegister_Click(self):
        self.destroyPackWidget(MainForm.main_Root)
        frmSignUp = SignUp(MainForm.main_Root)
        frmSignUp.pack()



        
class Login(Frame):
    main_Root=None
    def destroyPackWidget(self,parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        Login.main_Root=master
        super().__init__(master=master)
        master.title("Login")
        master.geometry("300x250")
        self.createWidget()
    def createWidget(self):
        self.lblMsg=Label(self, text="Please enter details below to login",bg="light blue")
        self.lblMsg.pack()
        self.username=Label(self, text="Username * ")
        self.username.pack()
        self.username_verify = StringVar()
        self.username_login_entry = Entry(self, textvariable=self.username_verify)
        self.username_login_entry.pack()
        self.password=Label(self, text="Password * ")
        self.password.pack()
        self.password_verify = StringVar()
        self.password_login_entry = Entry(self, textvariable=self.password_verify, show='*')
        self.password_login_entry.pack()
        self.btnLogin=Button(self, text="Login", width=10, height=1, command=self.btnLogin_Click)
        self.btnLogin.pack()
    def btnLogin_Click(self):
        username1 = self.username_login_entry.get()
        password1 = self.password_login_entry.get()
        
#        messagebox.showinfo("Failure", self.username1+":"+password1)
        list_of_files = os.listdir()
        if username1 in list_of_files:
            file1 = open(username1, "r")
            verify = file1.read().splitlines()
            if password1 in verify:
                messagebox.showinfo("Sucess","Login Sucessful")
                self.destroyPackWidget(Login.main_Root)
                frmQuestion = QuestionDigonosis(Login.main_Root)
                frmQuestion.pack()
            else:
                messagebox.showinfo("Failure", "Login Details are wrong try again")
        else:
            messagebox.showinfo("Failure", "User not found try from another user\n or sign up for new user")


class SignUp(Frame):
    main_Root=None
    print("SignUp Class")
    def destroyPackWidget(self,parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        SignUp.main_Root=master
        master.title("Register")
        super().__init__(master=master)
        master.title("Register")
        master.geometry("300x250")
        self.createWidget()
    def createWidget(self):
        self.lblMsg=Label(self, text="Please enter details below", bg="light blue")
        self.lblMsg.pack()
        self.username_lable = Label(self, text="Username * ")
        self.username_lable.pack()
        self.username = StringVar()
        self.username_entry = Entry(self, textvariable=self.username)
        self.username_entry.pack()

        self.password_lable = Label(self, text="Password * ")
        self.password_lable.pack()
        self.password = StringVar()
        self.password_entry = Entry(self, textvariable=self.password, show='*')
        self.password_entry.pack()
        self.btnRegister=Button(self, text="Register", width=10, height=1, bg="light blue", command=self.register_user)
        self.btnRegister.pack()


    def register_user(self):
        file = open(self.username_entry.get(), "w")
        file.write(self.username_entry.get() + "\n")
        file.write(self.password_entry.get())
        file.close()
        
        self.destroyPackWidget(SignUp.main_Root)
        
        self.lblSucess=Label(root, text="Registration Success", fg="green", font=("calibri", 11))
        self.lblSucess.pack()
        
        self.btnSucess=Button(root, text="Click Here to proceed", command=self.btnSucess_Click)
        self.btnSucess.pack()
    def btnSucess_Click(self):

        self.destroyPackWidget(SignUp.main_Root)
        frmQuestion = QuestionDigonosis(SignUp.main_Root)

        frmQuestion.pack()



root = Tk()

frmMainForm=MainForm(root)
frmMainForm.pack()
root.mainloop()




SignUp Class
